# Lecture Camera

In [5]:
import numpy as np # matrices
import cv2 # traitement d'images
import matplotlib.pyplot as plt # affichage

In [6]:
import cv2  
def list_ports():
    """
    Test the ports and returns a tuple with the available ports and the ones that are working.a
    """
    is_working = True
    dev_port = 0
    working_ports = []
    available_ports = []
    while is_working:
        camera = cv2.VideoCapture(dev_port)
        if not camera.isOpened():
            is_working = False
            print("Port %s is not working." %dev_port)
        else:
            is_reading, img = camera.read()
            w = camera.get(3)
            h = camera.get(4)
            if is_reading:
                print("Port %s is working and reads images (%s x %s)" %(dev_port,h,w))
                working_ports.append(dev_port)
            else:
                print("Port %s for camera ( %s x %s) is present but does not reads." %(dev_port,h,w))
                available_ports.append(dev_port)
        dev_port +=1
    return available_ports,working_ports

In [15]:

import cv2

def capture_image():
    ports_libres, ports_fonctionnels  = list_ports()
    cap = cv2.VideoCapture(0) 
    ret, img = cap.read()  

    # Vérifier si la capture d'image a été réussie
    if ret:
        # Afficher l'image capturée
        cv2.imshow("Image capturée", img)
        cv2.waitKey(0)
    else:
        print("Erreur lors de la capture de l'image")
        cap.release()
        cv2.destroyAllWindows()
        #gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Vérifiez que la caméra fonctionne correctement
    if not cap.isOpened():
        print("Impossible d'ouvrir la caméra")
        # Capturez une image et vérifiez qu'elle a été correctement chargée
        ret, img = cap.read()
        cap.release()
    if not ret:
        print("Impossible de capturer une image")
    else:
        #plt.figure()
        #plt.imshow(img)
        #plt.show()
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        plt.imshow(gray_img, cmap='gray')
        plt.show()  
    return img


Port 0 is not working.
Erreur lors de la capture de l'image
Impossible d'ouvrir la caméra
Impossible de capturer une image


ENVOIT DES COMMANDES SERIES A L'ARDUINO

In [14]:
import serial
import time
from enum import Enum

# Fonction du status des actions
def get_status():
    return send_command(100, 0)

# FONCTION PILOTAGE LED

def led_on():
    send_command(0, 0)

def led_off():
    send_command(1, 0)

# FONCTION D'ENVOIT 

def send_command_move(cmd, data):
    rep = send_command(cmd, data)
    if '1' in rep :
        in_progress = True
        while in_progress :
            time.sleep(0.1)
            rep = get_status()
            if '1' in rep :
                in_progress = False

def send_command(cmd, data):
    id = 0x55
    checksum = id + cmd + data
    nbr = ser.write([id,cmd,data,checksum])
    print("Send command : {0} - {1} / crc : {2}".format(cmd, nbr, checksum))

    rep = str(ser.readline())

    print(rep)
    return rep


ser = serial.Serial('COM5', 115200, timeout=0.1)
print(ser.name)



print("wait reset")
time.sleep(1)


# led off
led_off()
time.sleep(3)

# led on 
led_on()
time.sleep(3)


ser.close()

SerialException: could not open port 'COM5': FileNotFoundError(2, 'Le fichier spécifié est introuvable.', None, 2)

FONCTIONS BAS NIVEAU

In [13]:
# FONCTION PINCE FIXE : MOTEUR __ = avancement / MOTEUR __ = ouverture

def fix_grip_forward(number):
    send_command_move(2, number)

def fix_grip_backward():
    send_command_move(3, 0)

def fix_grip_open():
    send_command_move(4, 0)

def fix_grip_close():
    send_command_move(5, 0)


# FONCTIONS PINCE ROTATION : MOTEUR __ = avancement / MOTEUR __ = ouverture / MOTEUR __ = rotation

def grip_forward(number):
    send_command_move(6, number)

def grip_backward():
    send_command_move(7, 0)

def grip_open():
    send_command_move(8, 0)

def grip_close():
    send_command_move(9, 0)

def rotate_gripper(rotation):               # Définir le sens de rotation ! : --> roation du cube anti-horaire du point de vue de la pince
    tmp = angle_to_rotation_cmd(rotation)
    send_command_move(10, tmp)

def angle_to_rotation_cmd(angle):
    if angle == 90 :
        return 0
    elif angle == 180 :
        return 1
    elif angle == 270 :
        return 2
    
# FONCTIONS DU PLATEAU : MOTEUR __ = hauteur / MOTEUR __ = rotation
    
def move_plate_up():
    send_command_move(11, 0)

def move_plate_down():
    send_command_move(12, 0)

def rotate_plate(rotation):
    tmp = angle_to_rotation_cmd(rotation)
    send_command_move(13, tmp)


FONCTIONS MOYEN NIVEAU

In [12]:
def move_grippers(left_number): # Place les pinces et serre le cube à la ligne que l'on veut
    grip_open()
    fix_grip_open()
    grip_backward()
    fix_grip_backward()
    grip_forward(left_number)        
    grip_close()
    
    if left_number >= 1 and left_number <= 3 :
        fix_grip_forward(5 - left_number)
    else :
        raise Exception("Invalid number of rows")

    fix_grip_close()


def rotate_cube_vertical(rotation): # Fait une rotation de la face du bas qui repose sur le plateau
    move_plate_up()
    grip_open()
    fix_grip_open()
    grip_backward()
    fix_grip_backward()
    grip_forward(3)
    grip_close()
    rotate_plate(rotation)

def rotate_cube_horizontal(rotation): # Fait une rotation de tout le cube posé sur le plateau
    move_plate_up()
    grip_open()
    fix_grip_open()
    grip_backward()
    fix_grip_backward()
    rotate_plate(rotation)

def rotate_left(number, rotation): # Fait une rotation de la face serrée par la pince qui tourne (face à droite de la caméra / à gauche des spectateurs)
    move_plate_up()
    move_grippers(number)
    move_plate_down()
    rotate_gripper(rotation)

FONCTIONS DE HAUT NIVEAU

In [11]:
def rotate_couronne(number, rotation):
    if number > 0 and number <= 3 :
        rotate_left(number, rotation)
    else :
        rotate_cube_horizontal(180)
        rotate_left(6-number, 360 - rotation)
        rotate_cube_horizontal(180)

def rotate_top(angle):
    rotate_cube_vertical(90)
    rotate_plate(90)
    rotate_left(1, angle)
    rotate_plate(270)
    rotate_cube_vertical(270)

FONCTIONS CAMERA ET CALCULS DE COULEURS

In [10]:
R = 0   # RED
G = 1   # GREEN
B = 2   # BLUE
Y = 3   # YELLOW
O = 4   # ORANGE
W = 5   # WHITE

    
def rgb_to_color(red, green, blue): # Fonction qui converti les valeur d'un point R G B en une couleur 
    if red < 125 and red > 95 and green < 53 and green > 33 and blue < 44 and blue > 24 :
        return R
    elif red < 82 and red > 62 and green < 196 and green > 176 and blue < 65 and blue > 45 :
        return G
    elif red < 20 and red >= 0 and green < 68 and green > 48 and blue < 125 and blue > 105 :            # VALEURS DES COMPARAISONS RGB A AJUSTER SELON LUMIERE QUI ECLAIRE LE CUBE !!!!!!
        return B
    elif red < 170 and red > 150 and green < 183 and green > 163 and blue < 48 and blue > 28 :
        return Y
    elif red < 197 and red > 177 and green < 110 and green > 90 and blue < 47 and blue > 27 :
        return O
    elif red < 190 and red > 150 and green < 234 and green > 204 and blue < 237 and blue > 197 :
        return W
    else: raise Exception("error of conversion rgb to color")   

# FONCTION POUR CAPTURER LES IMAGES DE TOUTES LES FACES
def mouve_to_capture(case):
    if case == 0:
        rotate_cube_horizontal(90)
    elif case == 1:
        rotate_cube_horizontal(90)
    elif case == 2:
        rotate_cube_horizontal(90)
    elif case == 3:
        move_plate_up()         # En admettant que lors du démarage de la machine, les pinces sont ouvertes et reculée !!!!!! faire la séquence suivante : 
        grip_forward(2) 
        grip_close()
        move_plate_down()
        rotate_gripper(90)  #(face du haut vers la caméra)
        move_plate_up()
        grip_open()
        grip_backward()
        move_plate_down()
    elif case == 4:
        rotate_cube_horizontal(180)
    elif case == 5:
        #remettre le cube dans sa position d'origine
        rotate_cube_horizontal(180)
        move_plate_up()         
        grip_forward(2) 
        grip_close()
        move_plate_down()
        rotate_gripper(270)
        move_plate_up()
        grip_open()
        grip_backward()
        move_plate_down()
        rotate_cube_horizontal(90)

# FONCTIONS POUR CAPTURER UN POINT RGB SUR L'IMAGE CAPTUREE
def capture_pixel_color(x, y, img):

    # obtenez les valeurs RGB du point sélectionné
    blue = img.item(y, x, 0)
    green = img.item(y, x, 1)
    red = img.item(y, x, 2)

    # affichez les valeurs RGB
    #print("Valeurs RGB du point ({}, {}): ({}, {}, {})".format(x, y, blue, green, red))

    return red, green, blue

# FONCTION QUI CAPTURE PLUSIEURS PIXELS AUTOUR D'UN POINT POUR EN FAIRE LA MOYENE
def pixels_color_mean(x, y, img):
    # Variables qui somment les points de couleurs
    point_red, point_green, point_blue = 0
    red, green, blue = 0

    for ofset_x in range(3):
        for ofset_y in range(3):
            # Capture des points de couleur autour des coordonnées x, y
            point_red, point_green, point_blue = capture_pixel_color((x - 1) + ofset_x, (y - 1) + ofset_y, img)
            red += point_red
            green += point_green
            blue += point_blue

    red /= 9
    green /= 9
    blue /= 9
    # Transformation des valeurs RGB en une couleur
    color = rgb_to_color(red, green, blue)

    return color

# FONCTION QUI CAPTURE UNE FACE
def capture_colors_face(face, tbl_faces):
    coordonate_x = 625  # reviens à cette valeur quand on reviens dans la fonction ???
    coordonate_y = 100
    ofset_x = 188
    ofset_y = 178
    img = capture_image()

    for x in range(5):
        for y in range(5):
            tbl_faces[[x][y][face]] = pixels_color_mean(coordonate_x + x * ofset_x, coordonate_y + y * ofset_y, img)
    return tbl_faces

# FONCTION QUI CAPTURE TOUTES LES FACES DU CUBE
def capture_colors_cube(tbl_faces): #tableau 3 dim (1 : couleur carré en x 2 : )
    for num_face in range(6):
        # Appel de la fonction pour capturer la face (le cube ne bouge pas avant la 1ère capture)
        tbl_faces = capture_colors_face(num_face, tbl_faces)
        # Appel de la fonction pour bouger le cube et mettre la face suivante en face de la caméra
        mouve_to_capture(num_face)

    return tbl_faces
    

ALGORYTHME DE RESOLUTION / ENVOIT DES COMMANDES :

In [ ]:
send_command_move(11, 0)

PortNotOpenError: Attempting to use a port that is not open